In [7]:
import matplotlib
import PIL
import os

path = 'C:/Users/Mihail/Desktop/Учёба/ML/data/dice/dice_photoset_2/'
directory = os.fsencode(path + 'origin/')

delta = 220
delta2 = int(delta/2)
size1 = 3264
size1_half = int(size1/2)
size2 = 2448
size2_half = int(size2/2)
boxes1 = ((0, 0, size1, size2 - delta),
            (0, 0, size1_half, size2_half - delta2),
            (0, size2_half - delta2, size1_half, size2 - delta),
            (size1_half, 0, size1, size2_half - delta2),
            (size1_half, size2_half - delta2, size1, size2 - delta),)
boxes2 = ((0, 0, size2, size1 - delta),
            (0, 0, size2_half, size1_half - delta2),   
            (0, size1_half - delta2, size2_half, size1 - delta),
            (size2_half, 0, size2, size1_half - delta2),
            (size2_half, size1_half - delta2, size2, size1 - delta),)
for file in os.listdir(directory)[1:]:
    filename = os.fsdecode(file)
    img = PIL.Image.open(path + 'origin/' + filename)
    PIL.ImageOps.exif_transpose(img, in_place=True)
    if img.height == size2:
        boxes = boxes1
    elif img.height == size1:
        boxes = boxes2
    img2 = img.crop(boxes[0])
    img2.crop(boxes[1]).save('C:/Users/Mihail/Desktop/Учёба/ML/data/dice/dice_photoset_2/changed/' + filename.split('.')[0] + '_changed1.jpg')
    img2.crop(boxes[2]).save('C:/Users/Mihail/Desktop/Учёба/ML/data/dice/dice_photoset_2/changed/' + filename.split('.')[0] + '_changed2.jpg')
    img2.crop(boxes[3]).save('C:/Users/Mihail/Desktop/Учёба/ML/data/dice/dice_photoset_2/changed/' + filename.split('.')[0] + '_changed3.jpg')
    img2.crop(boxes[4]).save('C:/Users/Mihail/Desktop/Учёба/ML/data/dice/dice_photoset_2/changed/' + filename.split('.')[0] + '_changed4.jpg')

In [8]:
import matplotlib
import PIL
import os

directory = os.fsencode(path + 'changed')
image_size_dir = {}
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    img = PIL.Image.open('C:/Users/Mihail/Desktop/Учёба/ML/data/dice/dice_photoset_2/changed/' + filename)
    image_size_dir[filename] = list((img.width, img.height))

In [9]:
import pandas as pd
annotations_path2 = '../../data/dice/dice_photoset_2/'
annotations_filename = 'dice_set2_json.json'
annotations = pd.read_json(annotations_path2 + annotations_filename).T
annotations = annotations[annotations['regions'].str.len() != 0]
annotations.head()

,filename,size,regions,file_attributes
P_20240928_215338_changed1.jpg148273,P_20240928_215338_changed1.jpg,148273,"[{'shape_attributes': {'name': 'polygon', 'all...",{}
P_20240928_215338_changed2.jpg135360,P_20240928_215338_changed2.jpg,135360,"[{'shape_attributes': {'name': 'polygon', 'all...",{}
P_20240928_215338_changed3.jpg149662,P_20240928_215338_changed3.jpg,149662,"[{'shape_attributes': {'name': 'polygon', 'all...",{}
P_20240928_215338_changed4.jpg134454,P_20240928_215338_changed4.jpg,134454,"[{'shape_attributes': {'name': 'polygon', 'all...",{}
P_20240928_215349_changed1.jpg145585,P_20240928_215349_changed1.jpg,145585,"[{'shape_attributes': {'name': 'polygon', 'all...",{}


In [4]:
annotations.loc['P_20240928_215338_changed1.jpg148273', 'regions']

[{'shape_attributes': {'name': 'polygon',
   'all_points_x': [381,
    398,
    411,
    420,
    429,
    439,
    447,
    452,
    452,
    451,
    437,
    409,
    392,
    379,
    367,
    362,
    362,
    364],
   'all_points_y': [22,
    21,
    23,
    26,
    31,
    40,
    54,
    68,
    83,
    100,
    111,
    113,
    108,
    98,
    81,
    65,
    47,
    33]},
  'region_attributes': {'dice_value': '5'}},
 {'shape_attributes': {'name': 'polygon',
   'all_points_x': [992,
    1009,
    1025,
    1038,
    1045,
    1047,
    1042,
    1035,
    1028,
    1022,
    1006,
    980,
    970,
    960,
    959,
    963,
    970,
    977],
   'all_points_y': [289,
    294,
    303,
    313,
    324,
    336,
    349,
    367,
    378,
    382,
    380,
    369,
    358,
    346,
    334,
    319,
    305,
    295]},
  'region_attributes': {'dice_value': '4'}},
 {'shape_attributes': {'name': 'polygon',
   'all_points_x': [518,
    564,
    574,
    577,
    577,
    574,


In [228]:
from PIL import ImagePath, Image, ImageDraw
import numpy as np
image_size_dirr = pd.DataFrame(data=[image_size_dir.values(), image_size_dir.keys()]).T
image_size_dirr.columns = ['image_size', 'filename']
df = pd.DataFrame(data=annotations.values, columns=list(annotations.columns))
df = pd.merge(df, image_size_dirr)
output_df = pd.DataFrame(df['filename'])
output_df['coordinates'] = [None for i in df.index]
output_df['class_label'] = [None for i in df.index]
# df['masks'] = [[] for i in df.index]
masks = np.array([[] for i in df.index], dtype=int)
for i in df.index:
    cords = []
    labels = []
    for k,j in enumerate(df.loc[i, 'regions']):
        cords.append(tuple((min(j['shape_attributes']['all_points_x']), min(j['shape_attributes']['all_points_y']), max(j['shape_attributes']['all_points_x']), max(j['shape_attributes']['all_points_y']))))
        labels.append(j['region_attributes']['dice_value'])
        image = PIL.ImagePath.Path(list(zip(j['shape_attributes']['all_points_x'], j['shape_attributes']['all_points_y']))).getbbox()
        size = df.loc[i, 'image_size']
        img = Image.new("1", size, "white")  
        img1 = ImageDraw.Draw(img)   
        img1.polygon(list(zip(j['shape_attributes']['all_points_x'], j['shape_attributes']['all_points_y'])), fill ="black")
        img.save(path + 'masks/' + df.loc[i, 'filename'].split('.')[0] + '_mask' + str(k) + '.jpg')
        np.append(masks[i], img)
    output_df.at[i, 'coordinates'] = tuple(cords)
    output_df.at[i, 'class_label'] = tuple(labels)
np.save('masks.npy', masks, allow_pickle=True, fix_imports=True)
output_df.to_csv('processed_data.csv')
output_df.head()

,filename,coordinates,class_label
0,P_20240928_215338_changed1.jpg,"((360, 20, 456, 111), (474, 656, 587, 752), (5...","(5, 2, 1, 2, 6, 1, 1, 4, 5, 1, 6, 3)"
1,P_20240928_215338_changed2.jpg,"((1232, 156, 1328, 246), (980, 224, 1083, 322))","(6, 2)"
2,P_20240928_215338_changed3.jpg,"((80, 852, 153, 929), (207, 631, 311, 741), (3...","(5, 2, 3, 2, 4)"
3,P_20240928_215338_changed4.jpg,"((211, 37, 309, 131),)","(1,)"
4,P_20240928_215349_changed1.jpg,"((433, 1231, 505, 1296), (796, 1189, 880, 1274...","(3, 4, 2)"
5,P_20240928_215349_changed2.jpg,"((489, 49, 575, 131), (816, 7, 903, 91), (873,...","(4, 3, 1, 2, 3, 2, 5, 1, 3, 1, 3, 5, 2, 4, 5)"
6,P_20240928_215349_changed3.jpg,"((160, 1292, 242, 1378), (80, 1353, 159, 1427))","(6, 6)"
7,P_20240928_215349_changed4.jpg,"((4, 96, 89, 178), (275, 106, 357, 188), (548,...","(2, 3, 4, 5, 3, 3)"
8,P_20240928_215352_changed1.jpg,"((759, 1389, 845, 1471),)","(2,)"
9,P_20240928_215352_changed2.jpg,"((1457, 1, 1542, 77), (455, 749, 559, 851), (5...","(4, 3, 2, 6, 1, 1, 3, 4)"


In [10]:
from PIL import ImagePath, Image, ImageDraw
import torch
import numpy as np
image_size_dirr = pd.DataFrame(data=[image_size_dir.values(), image_size_dir.keys()]).T
image_size_dirr.columns = ['image_size', 'filename']
df = pd.DataFrame(data=annotations.values, columns=list(annotations.columns))
df = pd.merge(df, image_size_dirr)
for i in df.index:
    cords = []
    labels = []
    if_any_region_marked = 0 # КОСТЫЛЬ!!! УБРАТЬ ЛИШНИЕ РЕГИОНЫ С ПИКЧ!!!
    mask_id = 0
    for j in df.loc[i, 'regions']:
        if j['region_attributes']['dice_value'] == '':
            continue
        if_any_region_marked += 1
        cords.append(tuple((min(j['shape_attributes']['all_points_x']), min(j['shape_attributes']['all_points_y']), max(j['shape_attributes']['all_points_x']), max(j['shape_attributes']['all_points_y']))))
        labels.append(int(j['region_attributes']['dice_value']))
        image = PIL.ImagePath.Path(list(zip(j['shape_attributes']['all_points_x'], j['shape_attributes']['all_points_y']))).getbbox()
        size = df.loc[i, 'image_size']
        img = Image.new("1", size, "black")
        img1 = ImageDraw.Draw(img)   
        img1.polygon(list(zip(j['shape_attributes']['all_points_x'], j['shape_attributes']['all_points_y'])), fill ="white")
        img.save(path + 'masks/' + df.loc[i, 'filename'].split('.')[0] + '_mask' + str(mask_id) + '.jpg')
        mask_id += 1
    torch.save(torch.tensor(cords, dtype=torch.float), path + 'coordinates/' + df.loc[i, 'filename'].split('.')[0] + '_coordinates.torch')
    torch.save(torch.tensor(labels, dtype=torch.int64), path + 'labels/' + df.loc[i, 'filename'].split('.')[0] + '_labels.torch')
    if not if_any_region_marked:
        df = df.drop(index=i)
# df = df.reindex(axis=0)
pd.DataFrame(df['filename']).to_csv(path + 'filename.csv')

In [13]:
sum(df['filename'] == 'P_20240928_215400_changed2.jpg')

0

In [14]:
sum(df['filename'] == 'P_20240928_215400_changed2.jpg')

0

In [15]:
df[df['filename'] == 'P_20240928_215400_changed2.jpg'].index

Index([], dtype='int64')

In [66]:
df = df.drop(index=25)

In [214]:
output_df.to_json('processed_data.json')